# Plotting temperature data in the Southern Ocean
This script produces plots related to water temperature in the Southern Ocean using outputs from ACCESS-OM2-01.  
**Requirements:** It is suggested you use the `conda/analysis3-20.01` (or later) kernel. This can be defined using the drop down list on the left hand corner, or type `!module load conda/analysis3` in a Python cell.

## Loading relevant modules
These modules are used to access relevant outputs and to manipulate data. 

In [1]:
#This first line will show plots produced by matplotlib inside this Jupyter notebook
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import netCDF4 as nc
import xarray as xr
import numpy as np
import pandas as pd
import datetime as dt
import os
import calendar
from dask.distributed import Client
from tqdm import tqdm_notebook #to access observations
import gc #to free up memory
#Activate if needed
import copy

The following modules are used in map creation.

In [2]:
import cmocean as cm                              # Nice colormaps
from collections import OrderedDict               # We often use this to organise our experiments
import cftime                                     # In case you need to work with time axes
from glob import glob                             # If you need to search file systems
import cartopy.crs as ccrs                        # For making maps with different projections
import cartopy.feature as cft                     # For adding features to maps

## Accessing model outputs
Start a cluster that has multiple cores to work with. Remember that the number of cores cannot exceed the number of CPUs requested when accessing GADI.  
If the line below does not run, skip it. The result is that the job will not be parallelised, but the script will still run.

In [3]:
client = Client(n_workers = 12)

Access the default database of experiments from where data will be loaded.

In [4]:
session = cc.database.create_session()

This notebook uses the outputs for the v140 run of ACCESS-OM2-01 which includes wind forcing. Includes experiments `01deg_jra55v140_iaf` and `01deg_jra55v140_iaf`. A list of experiments can be accessed using `cc.querying.get_experiments(session)`, you can get a detailed list of experiments by adding `all = True` argument.

In [5]:
#Saving name of experiments of interest in variables that can be easily referred to
exp = "01deg_jra55v140_iaf_cycle2"

### Experiment variables
You can query the temporal frequency of all variables available in the experiment using `cc.querying.get_frequencies(session, experiment)`. A list of experiment variables can be obtained using `cc.querying.get_variables(session, experiment, frequency)`.

In [32]:
#Activate the lines below to query temporal frequency
cc.querying.get_frequencies(session, exp)

,frequency
0,None
1,1 daily
2,1 monthly
3,static


Query experiment variables list

In [16]:
#Querying experiment variables
expvar = cc.querying.get_variables(session, exp, frequency = "1 monthly")

#Extract variables with keywords included in its long_name column
expvar[expvar["long_name"].str.lower().str.match(".*temp.*")] #force all letters to be lowercase to get matches

#delete variable when no longer needed
#del expvar

,name,long_name,frequency,ncfile,# ncfiles,time_start,time_end
8,Tinz_m,ice internal temperatures on CICE grid,1 monthly,output487/ice/OUTPUT/iceh.2018-12.nc,60,2014-01-01 00:00:00,2019-01-01 00:00:00
9,Tsfc_m,snow/ice surface temperature,1 monthly,output487/ice/OUTPUT/iceh.2018-12.nc,732,1958-01-01 00:00:00,2019-01-01 00:00:00
32,diff_cbt_t,total vert diff_cbt(temp) (w/o neutral included),1 monthly,output487/ocean/ocean-3d-diff_cbt_t-1-monthly-...,244,1958-01-01 00:00:00,2019-01-01 00:00:00
83,pot_temp,Potential temperature,1 monthly,output487/ocean/ocean-3d-pot_temp-1-monthly-me...,244,1958-01-01 00:00:00,2019-01-01 00:00:00
107,surface_pot_temp,Potential temperature,1 monthly,output487/ocean/ocean-2d-surface_pot_temp-1-mo...,20,2014-01-01 00:00:00,2019-01-01 00:00:00
108,surface_pot_temp_min,Potential temperature,1 monthly,output487/ocean/ocean-2d-surface_pot_temp-1-mo...,20,2014-01-01 00:00:00,2019-01-01 00:00:00
110,surface_temp,Conservative temperature,1 monthly,output487/ocean/ocean-2d-surface_temp-1-monthl...,244,1958-01-01 00:00:00,2019-01-01 00:00:00
111,surface_temp_min,Conservative temperature,1 monthly,output487/ocean/ocean-2d-surface_temp-1-monthl...,244,1958-01-01 00:00:00,2019-01-01 00:00:00
118,temp,Conservative temperature,1 monthly,output487/ocean/ocean-3d-temp-1-monthly-mean-y...,244,1958-01-01 00:00:00,2019-01-01 00:00:00
119,temp_int_rhodz,vertical sum of Conservative temperature * rho...,1 monthly,output487/ocean/ocean-2d-temp_int_rhodz-1-mont...,244,1958-01-01 00:00:00,2019-01-01 00:00:00


## Accessing ACCESS-OM2 model outputs
Once the correct experiment variables have been identified, data can be loaded into the notebook for further processing. All variables needed to do this are included below.

In [6]:
#Name of experiment
exp
#Name (short name) of variable of interest as identified in previous step
varInt = 'surface_temp' #SST

Optional variables, activate if needed. Note that because times need correction, **the start time is actually one month after the month we are interested in**. See below for explanation.

In [135]:
#Give the start and end dates for the analyses. The input must be given as a list, even if it is one item only.
#Dates can be given as full date (e.g., 2010-01-01), just year and month, or just year. If multiple years are to be analysed, ensure both variables have the same length
#Start date
stime = [str(i)+'-01-10' for i in range(1958, 2019, 1)]
#End date
etime = [str(i)+'-12-20' for i in range(1958, 2019, 1)]
#Define frequency. Remember to check frequency and variable of interest are related to each other, for example, aice_m has a monthly frequency, while aice has a daily frequency.
freq = '1 monthly'

## Defining functions

**Accessing ACCESS-OM2-01 outputs**  
Defining function that loads data automatically using `cc.querying.getvar()` in a loop. The inputs needed are similar to those for the `cc.querying.getvar()` function, with the addition of inputs to define an area of interest.  
The `getACCESSdata` will achieve the following:  
- Access data for the experiment and variable of interest at the frequency requested and within the time frame specified  
- Transform temperature data from Kelvin to Celsius
- Minimum and maximum latitudes and longitudes can be specified in the function to access specific areas of the dataset if required.

The **Southern Ocean** is defined as ocean waters south of 45S.

In [136]:
#Frequency, experiment and session do not need to be specified if they were defined in the previous step
def getACCESSdata(var, start, end, minlon, maxlon, exp = exp, freq = freq, ses = session):
    #Accessing data
    vararray = cc.querying.getvar(exp, var, ses, frequency = freq, start_time = start, end_time = end)
    #Subsetting data to area of interest
    vararray = vararray.sel(yt_ocean = slice(minlon, maxlon))
    #Transform temperature from Kelvin to Celsius
    vararray = vararray-273.15
    return vararray

### Calculating SST anomalies
Anomalies are calculated per year and per season from the baseline. The time period chosen to calculate long-term mean SST, which will serve as a baseline, is between 1979 and 2008 (first 30 years of satellite records). Note that the calculation was performed once and the output saved to file. The baseline can be calculated as follows:  
`SO_SST = getACCESSdata(varInt, '1979-01-10, '2008-12-20', -90, -45, exp = exp, freq = freq, ses = session)` *Note that you can change the start and end dates to whatever time range is considered suitable for your analysis*
`baseSST = SO_SST.groupby('time.season').mean('time')`  
To save the output as a `.nc` file use the following line:  
`baseSST.to_netcdf('filepath.nc')` *Filepath includes the name of the file being saved.*  
To load the `.nc` file to the notebook use the following line:  
`SSTbaseline = xr.open_dataset('filepath.nc')` *Note that the filepath to open the dataset must be the same as the one provided to save the dataset.*

In [151]:
def SeasonalSSTMap(yrarray, basearray, **colPal):
    '''
    Additional arguments include: palette, colbar, inverse
    '''
    #Calling libraries to edit long and lat labels
    import matplotlib.ticker as mticker
    from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

    #Check if the 'palette' argument has been defined
    if 'palette' in colPal.keys():
        #If the 'palette' argument is a string
        if isinstance(colPal.get('palette'), str):
            #Defining colour palette from Scientific Colour Maps
            #Load relevant libraries to set Scientific Colour Map library
            from matplotlib.colors import LinearSegmentedColormap
            from matplotlib.colors import ListedColormap
            #Set path where the scientific library is found
            cm_data = np.loadtxt(os.path.join('../../ScientificColourMaps6', colPal['palette'], (colPal['palette'] + '.txt')))
            if 'inverse' not in colPal.keys() or colPal.get('inverse') != True:
                pal_map = LinearSegmentedColormap.from_list(colPal['palette'], cm_data)
            elif 'inverse' in colPal.keys() and colPal.get('inverse') == True:
                pal_map = ListedColormap(cm_data[::-1])
        #If the 'palette' argument is NOT a string
        elif not isinstance(colPal.get('palette'), str):
            #Load the colour palette defined in the 'palette' argument
            pal_map = colPal['palette']
    #If 'palette' argument has not been defined, then use cmocean ice palette as default
    else:
        pal_map = cm.cm.thermal

    #Calculate mean SST per season
    SeasonSST = yrarray.groupby('time.season').mean('time')

    #Calculate SST anomaly
    anomSST = SeasonSST - basearray

    #Defining season names
    seasonName = {'DJF': 'Austral summer', 'MAM': 'Austral autumn', 'JJA': 'Austral winter', 'SON': 'Austral spring'}

    #Change global font and font size
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.size'] = 14

    #Define the contour lines to be shown in the graph. From 0 to 1 and increasing by 0.005.
    levels = np.arange(-2.5, 2.51, 0.2)
    
    #Location of tickbars for colorbar
    cbar_ticks = [round(i, 1) for i in np.arange(-2.5, 2.51, 0.5)]

    #Define projection to polar
    projection = ccrs.SouthPolarStereo()

    #Create variable containing the Antarctic continent - 50m is medium scale
    land_50m = cft.NaturalEarthFeature('physical', 'land', '50m', edgecolor = 'black', facecolor = 'gray', linewidth = 0.5)

    #Create composite figures
    fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (16, 14), 
                             subplot_kw = dict(projection = projection))
    #Create counter using a list: i and j define the plot, while season refers to season
    for i, j, season in [(0, 0, 'DJF'), (0, 1, 'MAM'), (1, 0, 'JJA'), (1, 1, 'SON')]:
        #Extract information for each season and start plotting
        #pcolormesh creates a pseudo color map with a non-rectangular grid
        p1 = (anomSST.sel(season = season)).plot.contourf(x = 'xt_ocean', y = 'yt_ocean', ax = axes[i, j], 
                      #Setting scientific palette as color palette and defining min-max values for colourbar
                      cmap = pal_map, vmin = -2.5, vmax = 2.5, extend = 'both',
                      #Setting contour levels
                      levels = levels,
                      #Remove colourbar to allow for further manipulation
                      transform = ccrs.PlateCarree(), add_colorbar = False)
        #Set title for each subplot using season dictionaries. Move the title along the y axis so it is not too far from the plot
        axes[i,j].set_title(seasonName[season] + ' mean (months: ' + season + ')', y = 1.01)
        #Remove x and y axes labels
        axes[i,j].set_ylabel("")
        axes[i,j].set_xlabel("")

        #If the 'colbar' argument exist and it was set to 'each', then this section will be activated
        if 'colbar' in colPal.keys() and colPal.get('colbar').lower() == "each":
            #Set colourbar to be horizontal, 70% its original size and a distance equal to 9% of total figure area from the plot
            cb = fig.colorbar(p1, ax = axes[i,j], orientation = 'horizontal', shrink = 0.8, pad = 0.09, extend = 'both')
            #Set title for colourbar
            cb.ax.set_xlabel('Temperature ($^\circ$C)')
            #Setting colourbar ticks and labels
            cb.set_ticks(cbar_ticks)
            cb.set_ticklabels(cbar_ticks)

    #Changing other parameters
    for ax in axes.flat:
    #Coastlines to appear
        ax.coastlines(resolution = '50m')
    #Add feature loaded at the beginning of this section
        ax.add_feature(land_50m)
    #Set the extent of maps
        ax.set_extent([-280, 80, -80, -50], crs = ccrs.PlateCarree())
        #Adding gridlines gridlines - Removing any labels inside the map on the y axis
        gl = ax.gridlines(draw_labels = True, y_inline = False, color = "#b4b4b4", alpha = 0.6)
        #Locate longitude ticks - Set to every 30 deg
        gl.xlocator = mticker.FixedLocator(np.arange(-180, 181, 30))
        #Give longitude labels the correct format for longitudes
        gl.xformatter = LONGITUDE_FORMATTER
        #Set rotation of longitude labels to zero
        gl.xlabel_style = {'rotation': 0}
        #Set latitude labels to be transparent
        gl.ylabel_style = {'alpha': 0}
        #Add space between axis ticks and labels for x and y axes
        gl.xpadding = 8
        gl.ypadding = 8
        gl.xlabels_left = False
        gl.ylabels_right = False
        gl.xlabels_right = False
        gl.xlabels_top = False
        if ax.colNum == 0 and ax.rowNum == 1:
            gl.ylabels_left = True
            gl.xlabels_bottom = True
        elif ax.colNum == 0 and ax.rowNum != 1:
            gl.ylabels_left = True
            gl.xlabels_bottom = False
        elif ax.colNum != 0 and ax.rowNum == 1:
            gl.ylabels_left = False
            gl.xlabels_bottom = True
        elif ax.colNum != 0 and ax.rowNum != 1:
            gl.ylabels_left = False
            gl.xlabels_bottom = False

    #Set title for composite figure. Move the title along the y axis so it is not too far from 
    fig.suptitle('Mean sea surface temperature per season in ' + str(yrarray['time.year'].values[0]), y = 0.95, x = 0.45)

    #Decrease white space between each row of plots
    plt.subplots_adjust(hspace = 0.09, wspace = 0.05)

    #If the 'colbar' argument does not exist, or if it exist and it has not been set to 'each', then this section will be activated and one colourbar will be included in the figure
    if 'colbar' not in colPal.keys() or colPal.get('colbar').lower() != "each":
        #Change one colourbar for all figures
        #Set vertical colourbar for all subplots, 50% of original size and a distance equal to 2% of total figure area from the plot
        cb = fig.colorbar(p1, ax = axes[:,:], orientation = 'vertical', shrink = 0.5, pad = 0.02, extend = 'both')
        #Set title for colourbar
        cb.ax.set_ylabel('Temperature ($^\circ$C)')
        #Setting colourbar ticks and labels
        cb.set_ticks(cbar_ticks)
        cb.set_ticklabels(cbar_ticks)
        
    #Automatically saving figures to local folder
    plt.savefig(os.path.join('/g/data/v45/la6889/Figures/Maps/SeasonalSSTAnomaly/SeasonalSSTAnomaly'+ str(yrarray['time.year'].values[0]) + '.png'), dpi = 300)

## Applying functions
Loading SST data for the Southern Ocean

In [141]:
SO_SST = getACCESSdata(varInt, stime[0], etime[-1], -90, -45, exp = exp, freq = freq, ses = session)
SO_SST

<xarray.DataArray 'surface_temp' (time: 732, yt_ocean: 740, xt_ocean: 3600)>
dask.array<sub, shape=(732, 740, 3600), dtype=float32, chunksize=(1, 540, 720), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -45.18 -45.11 -45.04
  * time      (time) object 1958-01-16 12:00:00 ... 2018-12-16 12:00:00

Loading baseline dataset calculated previously

In [167]:
baseSST = xr.open_dataset('/g/data/v45/la6889/Calculations/BaselineSST_1979-2008.nc')
baseSST = baseSST['surface_temp']
baseSST

<xarray.DataArray 'surface_temp' (season: 4, yt_ocean: 740, xt_ocean: 3600)>
[10656000 values with dtype=float32]
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -45.18 -45.11 -45.04
  * season    (season) object 'DJF' 'JJA' 'MAM' 'SON'

Creating graphs of SST anomalies per season and per year

In [ ]:
for i, j in enumerate(stime):
    SSTyr = SO_SST.sel(time = stime[0].split('-')[0])
    SeasonalSSTMap(SSTyr, baseSST, palette = 'roma', inverse = True)

In [9]:
def SSTTimeSeries(array): 
    #Calculating min, mean and max sea ice extent per year
    Min = array.groupby('time.year').min('time')
    Mean = array.groupby('time.year').mean('time')
    Max = array.groupby('time.year').max('time')
    
    #Extracting dates information and turning them into pandas datetimes
    Dates = xr.cftime_range(start = str(array['time'].values.min().year), end = str(array['time'].values.max().year), freq = 'AS').to_datetimeindex()
    
    #Change global font and font size
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.size'] = 14

    #Extract information about the plot (figure and axes)
    fig, ax = plt.subplots(figsize = (16.4, 8.2))
    #Using Dates information to plot dates. Transforming area units from m2 to km2.
    plt.plot(Dates, Min, color = '#ddaa33', linestyle = '-', marker = 'o', markersize = 5)
    del Min
    plt.plot(Dates, Mean, color = '#bb5566', linestyle = '-', marker = 'o', markersize = 5)
    del Mean
    plt.plot(Dates, Max, color = '#004488', linestyle = '-', marker = 'o', markersize = 5)

    #Adding a main title to the plot - Increasing font size
    plt.title('Mean sea surface temperature in the Southern Ocean', fontsize = 16)

    #Changing y axes
    #Changing the limits of the y axis. Max value is 10% more than maximum value in the data
    plt.ylim(0, np.floor(max(Max).values*1.1))
    #Adding y label
    plt.ylabel('Temperature ($^\circ$C)')
    
    #Changing x axis
    #Change axes labels
    plt.xlabel('Year')
    #Extract min and max dates
    datemin = min(Dates)
    datemax = max(Dates)
    ax.xaxis.set_major_locator(mdates.YearLocator())
    #Change length of ticks - smaller for minor ticks
    ax.tick_params(axis = 'x', which = "major", length = 4)
    #Change the format of the dates shown (month abbreviation and year)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    
    #Set major gridlines to appear, but with high transparency
    ax.grid(True, color = "#b4b4b4", linestyle = '-', alpha = 0.2, linewidth = 0.5)
    
    #Automated path to save output to local folder
#     if min(Dates.year) != max(Dates.year):
#         plt.savefig(os.path.join('../Outputs/Figures/TimeSeries/SST', ('TimeSeriesSeaIce' + str(min(Dates.year)) + '-' + str(max(Dates.year)) + '.tiff')), dpi = 300)
#     else:
#         plt.savefig(os.path.join('../Outputs/Figures/TimeSeries/SST', ('TimeSeriesSeaIce' + str(min(Dates.year)) + '.tiff')), dpi = 300)
    
    #Return maximum value to set yearly y-axis
    return max(Max).values*1.1